# Drifting pixel pipeline

Here, drifting pixels are defined as pixels that are either almost constant over time (very low temporal variation) or whose median value over time deviates strongly from the local median of their neighborhood.  

The pipeline detects these pixels by calculating the temporal standard deviation and the local deviation from the median, and then creates a mask for "drifting pixels" (this can be saved as tiff if wanted)

Mode defines the desired band for which the mask should be produced

functions

In [6]:
import os
import glob
import numpy as np
import xarray as xr
from scipy.ndimage import median_filter
import rasterio


# load npy files and extract bands into xarray dataset

def load_frames_to_xarray(npy_folder):
    def extract_number(filename):
        # Extract the number from the filename, e.g., "frame_12.npy" -> 12
        basename = os.path.basename(filename)
        num = ''.join(filter(str.isdigit, basename))
        return int(num) if num else -1

    npy_files = sorted(glob.glob(os.path.join(npy_folder, "*.npy")), key=extract_number)
    # print("Read order:", npy_files)

    L2_list = []
    L1_list = []
    M0_list = []

    for f in npy_files:
        arr = np.load(f)
        L2 = arr[0:100, :, 0]      # L2: rows 0-100, band 0
        L1 = arr[349:419, :, 1]    # L1: rows 349-419, band 1
        M0 = arr[668:768, :, 2]    # M0: rows 668-767, band 2
        L2_list.append(L2)
        L1_list.append(L1)
        M0_list.append(M0)

    L2_arr = np.stack(L2_list)  # shape: (frame, x, y)
    L1_arr = np.stack(L1_list)
    M0_arr = np.stack(M0_list)

    ds = xr.Dataset(
        {
            "L2": (["frame", "x_L2", "y_L2"], L2_arr),
            "L1": (["frame", "x_L1", "y_L1"], L1_arr),
            "M0": (["frame", "x_M0", "y_M0"], M0_arr)
        },
        coords={
            "frame": np.arange(L2_arr.shape[0]),
            "x_L2": np.arange(L2_arr.shape[1]),
            "y_L2": np.arange(L2_arr.shape[2]),
            "x_L1": np.arange(L1_arr.shape[1]),
            "y_L1": np.arange(L1_arr.shape[2]),
            "x_M0": np.arange(M0_arr.shape[1]),
            "y_M0": np.arange(M0_arr.shape[2])
        }
    )
    return ds

# create drifting pixel mask based on temporal std and local median deviation
# mode: "L2", "L1" or "M0"
# save as tiff if wanted


def create_drifting_pixel_mask(ds, mode, out_path, transform = None):
    if mode == "L2":
        band_name = "L2"
        x_dim = "x_L2"
        y_dim = "y_L2"
    elif mode == "L1":
        band_name = "L1"
        x_dim = "x_L1"
        y_dim = "y_L1"
    elif mode == "M0":
        band_name = "M0"
        x_dim = "x_M0"
        y_dim = "y_M0"
    else:
        raise ValueError("Mode must be 'L2', 'L1' or 'M0'.")

    band = ds[band_name].values
    median_band = np.nanmedian(band, axis=0)
    local_median = median_filter(median_band, size=5)
    local_dev = np.abs(median_band - local_median)
    mad = np.nanmean(local_dev)
    outlier_mask_local = local_dev > 8 * mad
    temporal_std = np.nanstd(band, axis=0)
    const_mask = temporal_std < 1
    # checks if pixel is constant over time
    # or deviates strongly from local background (mad => mean of 5x5 neighborhood medians)

    # Drifting: constant or local outlier
    drifting_mask = (const_mask) | (outlier_mask_local)
    drifting_mask_float = drifting_mask.astype("float32") * 10000
    ds[f"drifting_pixel_mask_{band_name}"] = ([x_dim, y_dim], drifting_mask_float)

    drifting_out_path = os.path.join(out_path, f"drifting_pixel_mask_{band_name}.tif")
    with rasterio.open(
        drifting_out_path,
        "w",
        driver="GTiff",
        height=drifting_mask_float.shape[0],
        width=drifting_mask_float.shape[1],
        count=1,
        dtype="float32",
        transform=transform,
        crs="EPSG:4326"
    ) as dst:
        dst.write(drifting_mask_float, 1)
    print(f"Drifting pixel mask saved at: {drifting_out_path}")
    return drifting_mask_float

example pipeline

In [ ]:
npy_folder = "/frames"  # Replace with your npy files folder
out_path = "/output"  # Replace with your desired output folder
mode = "L2"  # Choose between "L2", "L1", or "M0"

ds = load_frames_to_xarray(npy_folder)
drifting_mask = create_drifting_pixel_mask(ds, mode, out_path, transform=None)